In [1]:
import numpy as np

In [3]:
# ?np.random.randn

## Calculate Wt function

$g(y_t|\alpha_t)= \mathcal{N}(y_t|0,b^2\exp^{\alpha_t})$

In [4]:
def logwgtfun():
    pass
def propagate(alpha,a,s2):
    """Returns new params"""
    pass

In [5]:
def bootstrap(y,N,a,s2,b2):
    n = len(y)
    particles = np.zeros((n,N))
    logWts = np.zeros((n,N))
    ests = np.zeros((n,2)) # Estimate of mean and varinace
    logWts[0,:] = logwgtfun(y[0], particles[0,:], b2)
    
    for i in range(n-1):
        # Resample
        # With replacement, probs are set to log weights
        ind = np.random.choice(N,size=N,  replace = True, prob = np.exp(logwgt[i,:])/np.sum(np.exp(logwgt[i,:])))
        # Propogate
        # i+1 is the new time-step
        particles[i+1,:] = propagate(particles[i,ind],a,s2,N)
        
        # Weight
        logwgt[i+1,:] = logwgtfun(y[i+1], particles[i+1,:], b2)
        
    return logwgt, particles

In [6]:
N= 100

logwgt, particles = bootstrapp(y,N,a,s2,b2)

In [7]:
# Draw estimates
ests = np.zeros((n,2))

for i in range(n):
    wgt = np.exp(logwgt[i,:])/ np.sum(np.exp(logwgt[i,:]))
    ests[i,0] = np.sum(wgt*particles[i,:]) # mean
    ests[i,1] = np.sum(wgt*particles[i,:]**2) - ests[i,0]
    

## EM ALgo

- 4 sufficient statistics to be estimated(look at notes!

In [8]:
def fizedLagSmoothing(y,N,a,s2,b2):
    n = len(y)
    particles = np.zeros((n,N))
    logWts = np.zeros((n,N))
    wgt = np.zeros((n,N))
    
    # Values at time t=0
    particles[0,:] = np.random.randn(N)
    logwgt[0,:] = logwgtfun(y[0], particles[0,:], b2)
    wgt[0,:] = np.exp(logwgt[0,:])/ np.sum(np.exp(logwgt[0,:]))
    
    ## Sufficient statistics
    T1=T2=T3=T4 = 0
    
    for i in range(n-1):
        # Resample
        # With replacement, probs are set to log weights
        ind = np.random.choice(N,size=N,  replace = True, prob = np.exp(logwgt[i,:])/np.sum(np.exp(logwgt[i,:])))
        particles = particles[:,ind] # copy particles
        # Propogate
        # i+1 is the new time-step
        particles[i+1,:] = propagate(particles[i,ind],a,s2,N) # Calc particle at t
        
        # Weight: Calc Weights at t
        logwgt[i+1,:] = logwgtfun(y[i+1], particles[i+1,:], b2)
        wgt[i+1,:] = np.exp(logwgt[i+1,:])/np.sum(np.exp(logwgt[i+1,:]))
        ## lag smoothing
        if i >= lag:
            T1 += np.sum(wgt[i+1]* particles[i+1-lag,:]**2)
            T2 += np.sum(wgt[i+1]* particles[i+1-lag,:]*particles[i-lag,:])
            T3 += np.sum(wgt[i+1]* particles[i-lag,:]**2)
            T4 += np.sum(wgt[i+1]*y[i-lag]**2 * np.exp(-particles[i-lag,:]))
    
    for i in range(lag):
        T1 += np.sum(wgt[i+1]* particles[i+1-lag,:]**2)
        T2 += np.sum(wgt[i+1]* particles[i+1-lag,:]*particles[i-lag,:])
        T3 += np.sum(wgt[i+1]* particles[i-lag,:]**2)
        T4 += np.sum(wgt[i+1]*y[i-lag]**2 * np.exp(-particles[i-lag,:]))
    T4 += np.sum(wgt[n-1]*y[n-1]**2 * np.exp(-particles[n-1,:])) # at T4 is from t=1, rest are from t=2
    
    return T1,T2,T3,T4

### Next Run EM Algo

In [ ]:
# No of iters is set
# Vectors to save variables in each iter is set
# variables are initialized

# Set N and lag

for i in range(N):
    # Get sufficient statistics
    # Use the formula to update a,s2,b2